In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import operator
from collections import Counter
from operator import itemgetter

In [2]:
df_small = pd.read_csv('../data/small_sample.csv')
df = pd.read_csv('../data/large_sample.csv')
df_small.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rating = np.ceil(df.rating)
df_small.rating = np.ceil(df_small.rating)

In [3]:
def create_count(comment):
    for x in comment.split(' '):
        if x in di:
            di[x.lower()] += 1
        else:
            di[x.lower()] = 1
    return di

In [4]:
%%time
di = dict()
df.loc[df.rating == 1].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
one_dict = di
di = dict()
df.loc[df.rating == 2].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
two_dict = di
di = dict()
df.loc[df.rating == 3].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
three_dict = di
di = dict()
df.loc[df.rating == 4].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
four_dict = di
di = dict()
df.loc[df.rating == 5].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
five_dict = di
di = dict()
df.loc[df.rating == 6].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
six_dict = di
di = dict()
df.loc[df.rating == 7].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
seven_dict = di
di = dict()
df.loc[df.rating == 8].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
eight_dict = di
di = dict()
df.loc[df.rating == 9].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
nine_dict = di
di = dict()
df.loc[df.rating == 10].loc[df.comment.apply(lambda x: len(x.split(' ')))<30].comment.apply(create_count)
ten_dict = di

CPU times: user 43.3 s, sys: 650 ms, total: 44 s
Wall time: 45.4 s


In [5]:
%%time
one_counter = Counter(one_dict)
two_counter = Counter(two_dict)
three_counter = Counter(three_dict)
four_counter = Counter(four_dict)
five_counter = Counter(five_dict)
six_counter = Counter(six_dict)
seven_counter = Counter(seven_dict)
eight_counter = Counter(eight_dict)
nine_counter = Counter(nine_dict)
ten_counter = Counter(ten_dict)

CPU times: user 34.9 ms, sys: 6.53 ms, total: 41.4 ms
Wall time: 43.9 ms


In [6]:
all_counter = one_counter + two_counter + three_counter + four_counter + five_counter + six_counter + seven_counter + eight_counter + nine_counter + ten_counter

In [7]:
all_dict = dict(Counter(all_counter))

In [8]:
%%time
new_data = {k: v for k, v in all_dict.items() if v > 5}

CPU times: user 25 ms, sys: 1.26 ms, total: 26.3 ms
Wall time: 25.6 ms


In [9]:
%%time
sortedList=sorted(new_data.values())
sorted_dict = dict()
for sortedKey in sortedList:
    for key, value in new_data.items():
        if value==sortedKey:
            sorted_dict[key]=value

CPU times: user 1min 40s, sys: 615 ms, total: 1min 40s
Wall time: 1min 43s


In [10]:
%%time
dict_list = []
for key, value in sorted_dict.items():
    temp = [key,value]
    dict_list.append(temp)

CPU times: user 38.6 ms, sys: 12.5 ms, total: 51.1 ms
Wall time: 50.1 ms


In [11]:
testing_df = df.loc[df.comment.apply(lambda x: len(x.split(' ')))<30].copy()

In [12]:
def make_it_lower(comment):
    new_comment = []
    for x in comment.split(' '):
        new_comment.append(x.lower())
    return ' '.join(new_comment)

In [13]:
%%time
testing_df['lower_comment'] = testing_df.comment.apply(lambda x: make_it_lower(x))
testing_df.drop(columns=['comment'], inplace=True)
testing_df.rename(columns={'lower_comment':'comment'}, inplace=True)

CPU times: user 3.76 s, sys: 116 ms, total: 3.88 s
Wall time: 4.12 s


In [14]:
def word_count(comment):
    if word[0] in comment.split(' '):
        if word[0] in number:
            number[word[0]] += comment.split(' ').count(word[0])
        else:
            number[word[0]] = comment.split(' ').count(word[0])

In [15]:
%%time
def make_report(w,z):
    labels = ['word', 'average/std', 'percent_average', 'percent_std', 'rating', 'word_count', 'divisor', 'percent']
    to_return = pd.DataFrame(columns=labels)
    error = []
    for word in dict_list[int(w):int(z)]:
        print(word)
        try:
            to_append = []
            std = 0
            average = 0
            for y in range(1,11):
                print(y)
                number = dict()
                testing_df.loc[testing_df.rating == y].comment.apply(word_count, word=word, number=number)
                divisor = testing_df.loc[df.rating == y].comment.count()
                percent = number[word[0]]/divisor
                to_append.append([word[0], None, None, None, y, number[word[0]], divisor, percent])
#             print(to_append)
            for_array = []
            for value in to_append:
                for_array.append(value[7])
            x = np.array(for_array)
            array1 = [[word[0], (np.sum(x)/10)/(np.std(x)), np.sum(x)/10, np.std(x), None, None, None, None]]
            for x in to_append:
                array1.append(x)
            inbetween_df = pd.DataFrame.from_records(array1, columns=labels)
#             print('not working')
#             print(inbetween_df.head())
            print(to_return.head())
            to_return = pd.concat([to_return, inbetween_df])
#             print('inside')
        except KeyError:
            error.append(word[0])
            continue
    return to_return, error

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11 µs


In [16]:
%%time
report = make_report(0,100)

['denominator', 6]
1


TypeError: word_count() got an unexpected keyword argument 'word'